In [1]:
# pip install googletrans

In [2]:
import os
import pickle 
import pandas as pd
import jieba
from zhon.hanzi import punctuation
import re
import googletrans
from pprint import pprint
import matplotlib.pyplot as plt 
from datetime import datetime

import warnings
warnings.filterwarnings("ignore")

# 1. 載入Corpus

In [3]:
# path='C:/Users/USER/Desktop/Prof_Hsieh_Project/about_emotion/Chinese_derivative/corpus'
# os.chdir(path)

# # Facebook posts
# fb_data = pd.read_pickle('facebook_to_1208.pkl')
# fb_data['Publication_Date'] = fb_data['Publication_Time'].apply(lambda x: str(x.date()))
# fb_data = fb_data.drop(columns=['Publisher', 'Post_url', 'Likes',
#        'Reactions', 'Comments', 'Shares', 'Cover_photo','Publication_Time'])
# fb_data['Publication_Date']= pd.to_datetime(fb_data['Publication_Date'])

path='C:/Users/user/Desktop/Chinese_derivative_1229/01_Data(news)'
os.chdir(path)

# News
news_data1 = pd.read_pickle('2022-10-29_NewsData.pkl')
news_data1.rename(columns = {'pub_date':'Publication_Time', 'content':'Post_Content'}, inplace = True)
news_data1['Publication_Date'] = news_data1['Publication_Time'].apply(lambda x: str(x.date()))
news_data1 = news_data1.drop(columns=['title','search_key', 'link', 'search_time', 'description','source','Publication_Time'])
news_data1['Publication_Date']= pd.to_datetime(news_data1['Publication_Date'])

news_data2 = pd.read_pickle('news_to_1208.pkl')
news_data2.rename(columns = {'pub_date':'Publication_Time', 'content':'Post_Content'}, inplace = True)
news_data2['Publication_Date'] = news_data2['Publication_Time'].apply(lambda x: str(x.date()))
news_data2 = news_data2.drop(columns=['title','search_key', 'link', 'search_time', 'description','source','Publication_Time'])
news_data2['Publication_Date']= pd.to_datetime(news_data2['Publication_Date'])

# 2. Preprocess

In [4]:
def preprocess(text):
    """
    remove punctuations, numbers, english letters, line break and blank.
    """
    for i in punctuation:
        text = text.replace(i, '')
    text = re.sub('[\d]','',text) # [0-9]
    
    text = re.sub(r'[.,"\'-?:!;]', '', text)
    
    text = re.sub('[a-zA-Z]','', text)
    
    text = text.replace("\n", "")
    text = text.replace(" ", "")
    return text

def tokenization(text): # 斷詞
    seg_list = jieba.cut(text, cut_all=False)
    return list(seg_list)

In [5]:
news_extract1 = news_data1.copy()
news_extract1['prepreocessed'] = news_extract1['Post_Content'].map(preprocess)
news_extract1['tokens'] = news_extract1['prepreocessed'].map(tokenization)
news_extract1 = news_extract1.drop(columns=['Post_Content','prepreocessed'])
news_extract1

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\user\AppData\Local\Temp\jieba.cache
Loading model cost 2.368 seconds.
Prefix dict has been built successfully.


,Publication_Date,tokens
0,2022-11-03,"[鴻海, 今日, 晚間, 宣布, 攜手, 沙烏地, 阿拉伯, 公共, 投資, 基金, 合資,..."
1,2022-11-03,"[財訊, 快報, 記者, 劉居全, 報導針, 對, 市場, 傳出, 中國, 已, 下令, 對..."
2,2022-11-02,"[河南, 鄭州, 疫情, 在, 月, 中旬, 擴散鴻海, 集團, 旗下, 富士康, 鄭州廠,..."
3,2022-11-03,"[鴻海, 旗下, 富智康, 日, 自結, 第季, 稅後虧, 損萬, 美元, 較, 去年同期,..."
4,2022-11-02,"[關注台, 股盤, 前, 要, 聞, 重點, 中國, 鄭州, 航空港, 經濟, 綜合, 實驗..."
...,...,...
2793,2022-11-03,"[信驊, 大立, 光旭, 隼, 這些, 高價股, 通常, 是, 法人, 的, 投資標, 的,..."
2794,2022-11-01,"[台股, 持續, 震盪, 有, 投資人, 紛紛, 出場, 止損, 也, 有人, 逢, 低, ..."
2795,2022-11-02,"[月, 中旬, 即將, 配息, 的, 公告, 出爐, 總, 共有, 檔將, 在, 日, 同步..."
2796,2022-11-01,"[隨著, 近期, 股市, 低迷, 投資, 和, 等, 的, 投資人, 不, 減反, 增獨, ..."


In [6]:
news_extract2 = news_data2.copy()
news_extract2['prepreocessed'] = news_extract2['Post_Content'].map(preprocess)
news_extract2['tokens'] = news_extract2['prepreocessed'].map(tokenization)
news_extract2 = news_extract2.drop(columns=['Post_Content','prepreocessed'])
news_extract2

,Publication_Date,tokens
0,2022-11-03,"[鴻海, 今日, 晚間, 宣布, 攜手, 沙烏地, 阿拉伯, 公共, 投資, 基金, 合資,..."
1,2022-11-03,"[財訊, 快報, 記者, 劉居全, 報導針, 對, 市場, 傳出, 中國, 已, 下令, 對..."
2,2022-11-02,"[河南, 鄭州, 疫情, 在, 月, 中旬, 擴散鴻海, 集團, 旗下, 富士康, 鄭州廠,..."
3,2022-11-03,"[鴻海, 旗下, 富智康, 日, 自結, 第季, 稅後虧, 損萬, 美元, 較, 去年同期,..."
4,2022-11-02,"[關注台, 股盤, 前, 要, 聞, 重點, 中國, 鄭州, 航空港, 經濟, 綜合, 實驗..."
...,...,...
17533,2022-12-15,"[產險, 防疫, 險理, 賠, 失血, 金額, 近尾聲, 資產重, 分類, 也將, 挹注, ..."
17534,2022-12-13,"[▲, 中環, 子公司, 賣陽明, 股票, 投資, 報酬, 率, %, 法人, 評落後大盤,..."
17535,2022-12-14,"[面對, 通膨時代, 退休金, 準備, 多少, 才夠, 中經院, 調查, 未, 退休, 族群..."
17536,2022-12-12,"[定存, 是, 最, 簡單, 的, 理財, 方式, 不, 需面, 臨虧, 損風險, 但, 相..."


In [7]:
# fb_extract = fb_data.copy()
# fb_extract['prepreocessed'] = fb_extract['Post_Content'].map(preprocess)
# fb_extract['tokens'] = fb_extract['prepreocessed'].map(tokenization)
# fb_extract = fb_extract.drop(columns=['Post_Content','prepreocessed'])
# fb_extract

# 3. Group to same date

In [8]:
news_condense = news_extract2.groupby("Publication_Date").agg({'tokens':'sum'})
# fb_condense = fb_extract.groupby("Publication_Date").agg({'tokens':'sum'})

In [9]:
news_condense = news_condense.reset_index()
# fb_condense = fb_condense.reset_index()

In [10]:
news_condense

,Publication_Date,tokens
0,2022-10-29,"[護國, 神山, 台積電, 近日, 內憂, 外患, 不斷, 市值, 也, 跌, 到, 兆, ..."
1,2022-10-30,"[本周, 迎來, 月, 的, 第一周, 鴻海, 預計將, 公布, 月, 營收, 由, 於, ..."
2,2022-10-31,"[聯發, 科上, 週五法, 說會釋, 出, 今年, 展望, 恐低, 於, 預期, 所幸, 已..."
3,2022-11-01,"[外資, 今日, 賣, 超台, 股, 億元, 由, 買, 轉賣, 台, 指期, 現貨淨, 多..."
4,2022-11-02,"[河南, 鄭州, 疫情, 在, 月, 中旬, 擴散鴻海, 集團, 旗下, 富士康, 鄭州廠,..."
5,2022-11-03,"[鴻海, 今日, 晚間, 宣布, 攜手, 沙烏地, 阿拉伯, 公共, 投資, 基金, 合資,..."
6,2022-11-04,"[鴻海進, 軍電動車, 再, 傳, 好消息, 日晚, 間鴻海, 發布, 與, 沙烏地, 阿拉..."
7,2022-11-05,"[我, 曾經, 在, 此, 專欄, 提過, 資金, 控管, 永遠, 是, 投資, 最, 重要..."
8,2022-11-06,"[台股, 進入, 財報, 法, 說, 會, 旺季, 本周, 聯詠鴻海國, 巨儒鴻等, 電子,..."
9,2022-11-07,"[蘋果, 公司, 與鴻海, 今日, 早上, 同步, 發出, 聲明, 顯示, 在, 紅色供, ..."


In [12]:
# tmp1 = news_condense.copy()
# tmp2 = fb_condense.copy()
# tmp1 = tmp1.merge(tmp2, on='Publication_Date', how='left')
# combine_df = tmp1.copy()

combine_df = news_condense.copy()

# sz = combine_df.shape[0]
# combine_df['Tokens'] = ''
# for i in range(sz):
#     try:
#         combine_df['Tokens'][i] = combine_df['tokens_x'][i] + combine_df['tokens_y'][i]
#     except: 
#         combine_df['Tokens'][i] = combine_df['tokens_x'][i]
        
# combine_df = combine_df.drop(columns=['tokens_x', 'tokens_y'])
combine_df['token_num'] = combine_df['tokens'].map(len)
combine_df

,Publication_Date,tokens,token_num
0,2022-10-29,"[護國, 神山, 台積電, 近日, 內憂, 外患, 不斷, 市值, 也, 跌, 到, 兆, ...",54726
1,2022-10-30,"[本周, 迎來, 月, 的, 第一周, 鴻海, 預計將, 公布, 月, 營收, 由, 於, ...",104586
2,2022-10-31,"[聯發, 科上, 週五法, 說會釋, 出, 今年, 展望, 恐低, 於, 預期, 所幸, 已...",267642
3,2022-11-01,"[外資, 今日, 賣, 超台, 股, 億元, 由, 買, 轉賣, 台, 指期, 現貨淨, 多...",192923
4,2022-11-02,"[河南, 鄭州, 疫情, 在, 月, 中旬, 擴散鴻海, 集團, 旗下, 富士康, 鄭州廠,...",261733
5,2022-11-03,"[鴻海, 今日, 晚間, 宣布, 攜手, 沙烏地, 阿拉伯, 公共, 投資, 基金, 合資,...",396320
6,2022-11-04,"[鴻海進, 軍電動車, 再, 傳, 好消息, 日晚, 間鴻海, 發布, 與, 沙烏地, 阿拉...",81034
7,2022-11-05,"[我, 曾經, 在, 此, 專欄, 提過, 資金, 控管, 永遠, 是, 投資, 最, 重要...",29568
8,2022-11-06,"[台股, 進入, 財報, 法, 說, 會, 旺季, 本周, 聯詠鴻海國, 巨儒鴻等, 電子,...",58546
9,2022-11-07,"[蘋果, 公司, 與鴻海, 今日, 早上, 同步, 發出, 聲明, 顯示, 在, 紅色供, ...",141240


In [13]:
combine_df.describe()

,token_num
count,48.000000
mean,149155.937500
std,102284.151082
min,24577.000000
25%,70706.500000
50%,141309.500000
75%,224674.250000
max,396320.000000


# Sentiment Analysis

In [15]:
path = 'C:/Users/user/Desktop/Chinese_derivative_1229/09_words_and_labels'
os.chdir(path)

Sentiment_Score = pd.read_pickle('Sentiment_Score_onlyOF.pkl')
labels = list(Sentiment_Score['token'])

In [16]:
Sentiment_Score

,token,market_sentiment
0,極端主義者,1.0
1,品嚐,1.0
2,力量,1.0
3,配件,1.0
4,極好的,1.0
...,...,...
933,堅定,-0.5
934,飢餓的,-0.5
935,盛行,-0.5
936,進攻,-0.5


In [17]:
def Average(lst):
    return sum(lst) / len(lst)

def Count_Sentiment_Score(input_tokens, show=False):
    score = 0
    count = 0
    for token in input_tokens:
        if token in labels:
            same_token_list = list(Sentiment_Score[Sentiment_Score['token'] == token]['market_sentiment'])
            if show:
                print(token, same_token_list)
            avg_score = Average(same_token_list)
            count += 1
            score += avg_score
    if(count == 0): return 0
    else: return score/count

In [18]:
Count_Sentiment_Score(combine_df['tokens'][0])

0.19820776553991237

In [19]:
combine_df['Sentiment_Score'] = combine_df['tokens'].map(Count_Sentiment_Score)

In [20]:
combine_df

,Publication_Date,tokens,token_num,Sentiment_Score
0,2022-10-29,"[護國, 神山, 台積電, 近日, 內憂, 外患, 不斷, 市值, 也, 跌, 到, 兆, ...",54726,0.198208
1,2022-10-30,"[本周, 迎來, 月, 的, 第一周, 鴻海, 預計將, 公布, 月, 營收, 由, 於, ...",104586,0.177651
2,2022-10-31,"[聯發, 科上, 週五法, 說會釋, 出, 今年, 展望, 恐低, 於, 預期, 所幸, 已...",267642,0.156203
3,2022-11-01,"[外資, 今日, 賣, 超台, 股, 億元, 由, 買, 轉賣, 台, 指期, 現貨淨, 多...",192923,0.189762
4,2022-11-02,"[河南, 鄭州, 疫情, 在, 月, 中旬, 擴散鴻海, 集團, 旗下, 富士康, 鄭州廠,...",261733,0.238091
5,2022-11-03,"[鴻海, 今日, 晚間, 宣布, 攜手, 沙烏地, 阿拉伯, 公共, 投資, 基金, 合資,...",396320,0.213031
6,2022-11-04,"[鴻海進, 軍電動車, 再, 傳, 好消息, 日晚, 間鴻海, 發布, 與, 沙烏地, 阿拉...",81034,0.260547
7,2022-11-05,"[我, 曾經, 在, 此, 專欄, 提過, 資金, 控管, 永遠, 是, 投資, 最, 重要...",29568,0.196464
8,2022-11-06,"[台股, 進入, 財報, 法, 說, 會, 旺季, 本周, 聯詠鴻海國, 巨儒鴻等, 電子,...",58546,0.226030
9,2022-11-07,"[蘋果, 公司, 與鴻海, 今日, 早上, 同步, 發出, 聲明, 顯示, 在, 紅色供, ...",141240,0.213626


In [21]:
combine_df.to_pickle('Sentiment_Analysis_Result_onlyOF.pkl')

In [ ]:
# combine_df.to_pickle('Sentiment_Analysis_Result.pkl')